In [42]:
# Objectif : Installer les dépendances et vérifier le GPU
!pip install kaggle torch torchvision opencv-python pandas matplotlib seaborn albumentations -q  # Installation silencieuse
!pip install onnx -q  # Pour l'exportation du modèle

# Vérifier que le GPU T4 est actif
!nvidia-smi  # Doit afficher Tesla T4

# Importer les bibliothèques nécessaires
import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import torch
from datetime import datetime
import time
from sklearn.model_selection import train_test_split
import shutil

# Définir un chemin pour sauvegarder les résultats
output_dir = '/content/drive/MyDrive/yolov9_results'
os.makedirs(output_dir, exist_ok=True)  # Créer le dossier si inexistant


Sat Apr 26 14:06:09 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   44C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [43]:
from google.colab import drive

# Check if the drive is already mounted
if not os.path.exists('/content/drive'):
  drive.mount('/content/drive')
else:
  print("Google Drive is already mounted!")

Google Drive is already mounted!


In [44]:
# Objectif : Télécharger le dataset via l'API Kaggle
# Étape 1 : Uploader kaggle.json (téléchargez-le depuis Kaggle : Profil > Settings > Create New API Token)
from google.colab import files
files.upload()  # Sélectionnez kaggle.json

# Étape 2 : Configurer kaggle.json
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle (2).json


In [45]:
# Cellule 3 : Télécharger et décompresser le dataset
# Dataset : https://www.kaggle.com/datasets/andrewmvd/dog-and-cat-detection
!kaggle datasets download -d andrewmvd/dog-and-cat-detection
!unzip -q dog-and-cat-detection.zip -d /content/dataset

# Vérifier que le dataset a été correctement décompressé
dataset_path = '/content/dataset'
if os.path.exists(dataset_path):
    print("Contenu du dossier dataset :")
    for root, dirs, files in os.walk(dataset_path):
        print(f"Dossier : {root}")
        print(f"Fichiers : {files[:5]}")
        print(f"Sous-dossiers : {dirs}")
        print("-" * 50)
else:
    raise FileNotFoundError("Le dossier /content/dataset n'a pas été créé. Vérifiez le téléchargement et la décompression.")

Dataset URL: https://www.kaggle.com/datasets/andrewmvd/dog-and-cat-detection
License(s): CC-BY-SA-4.0
dog-and-cat-detection.zip: Skipping, found more recently modified local copy (use --force to force download)
replace /content/dataset/annotations/Cats_Test0.xml? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
Contenu du dossier dataset :
Dossier : /content/dataset
Fichiers : ['classes.txt']
Sous-dossiers : ['labels', 'images', 'annotations']
--------------------------------------------------
Dossier : /content/dataset/labels
Fichiers : ['Cats_Test87.xml', 'Cats_Test3411.xml', 'Cats_Test105.xml', 'Cats_Test1768.xml', 'Cats_Test2076.xml']
Sous-dossiers : ['train', 'val', 'test']
--------------------------------------------------
Dossier : /content/dataset/labels/train
Fichiers : ['Cats_Test3244.txt', 'Cats_Test2396.txt', 'Cats_Test2570.txt', 'Cats_Test3620.txt', 'Cats_Test589.txt']
Sous-dossiers : []
--------------------------------------------------
Dossier : /content/dataset/labels/val
Fichie

In [46]:
# Objectif : Convertir les annotations .xml (Pascal VOC) en .txt (YOLO format)
import os
import xml.etree.ElementTree as ET
import shutil

# Définir les chemins
xml_dir = '/content/dataset/annotations'  # Dossier contenant les fichiers .xml
images_base_path = '/content/dataset/images'
labels_base_path = '/content/dataset/labels'

# Classes définies dans classes.txt
classes = ['cat', 'dog']

# Fonction pour convertir un fichier .xml en .txt (format YOLO)
def convert_voc_to_yolo(xml_path, output_path, classes, img_width, img_height):
    tree = ET.parse(xml_path)
    root = tree.getroot()

    out_lines = []
    for obj in root.findall('object'):
        name = obj.find('name').text
        if name not in classes:
            continue
        class_id = classes.index(name)
        bndbox = obj.find('bndbox')
        xmin = float(bndbox.find('xmin').text)
        xmax = float(bndbox.find('xmax').text)
        ymin = float(bndbox.find('ymin').text)
        ymax = float(bndbox.find('ymax').text)

        # Convertir en coordonnées normalisées pour YOLO
        x_center = (xmin + xmax) / 2.0 / img_width
        y_center = (ymin + ymax) / 2.0 / img_height
        width = (xmax - xmin) / img_width
        height = (ymax - ymin) / img_height

        out_lines.append(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

    with open(output_path, 'w') as f:
        f.writelines(out_lines)

# Parcourir les sous-dossiers train/val/test pour associer les images aux annotations
for split in ['train', 'val', 'test']:
    img_dir = os.path.join(images_base_path, split)
    label_dir = os.path.join(labels_base_path, split)

    # S'assurer que les dossiers existent
    os.makedirs(label_dir, exist_ok=True)

    # Parcourir les images dans chaque sous-dossier
    for img_file in os.listdir(img_dir):
        if img_file.endswith('.png'):
            img_path = os.path.join(img_dir, img_file)
            xml_file = img_file.replace('.png', '.xml')
            xml_path = os.path.join(xml_dir, xml_file)

            # Vérifier si le fichier .xml correspondant existe
            if os.path.exists(xml_path):
                # Charger l'image pour obtenir ses dimensions
                img = cv2.imread(img_path)
                img_height, img_width, _ = img.shape

                # Définir le chemin de sortie pour le fichier .txt
                txt_file = img_file.replace('.png', '.txt')
                txt_path = os.path.join(label_dir, txt_file)

                # Convertir l'annotation
                convert_voc_to_yolo(xml_path, txt_path, classes, img_width, img_height)
                print(f"Converti {xml_file} en {txt_file} dans {split}")
            else:
                print(f"Annotation manquante pour {img_file} dans {split}")

# Vérifier le contenu des dossiers labels/train, labels/val, labels/test
for split in ['train', 'val', 'test']:
    label_dir = os.path.join(labels_base_path, split)
    txt_files = [f for f in os.listdir(label_dir) if f.endswith('.txt')]
    print(f"Dossier {label_dir} : {len(txt_files)} fichiers .txt")

Converti Cats_Test510.xml en Cats_Test510.txt dans train
Converti Cats_Test1638.xml en Cats_Test1638.txt dans train
Converti Cats_Test3463.xml en Cats_Test3463.txt dans train
Converti Cats_Test1089.xml en Cats_Test1089.txt dans train
Converti Cats_Test1506.xml en Cats_Test1506.txt dans train
Converti Cats_Test1178.xml en Cats_Test1178.txt dans train
Converti Cats_Test2031.xml en Cats_Test2031.txt dans train
Converti Cats_Test532.xml en Cats_Test532.txt dans train
Converti Cats_Test375.xml en Cats_Test375.txt dans train
Converti Cats_Test3502.xml en Cats_Test3502.txt dans train
Converti Cats_Test1565.xml en Cats_Test1565.txt dans train
Converti Cats_Test2683.xml en Cats_Test2683.txt dans train
Converti Cats_Test501.xml en Cats_Test501.txt dans train
Converti Cats_Test534.xml en Cats_Test534.txt dans train
Converti Cats_Test1790.xml en Cats_Test1790.txt dans train
Converti Cats_Test2422.xml en Cats_Test2422.txt dans train
Converti Cats_Test2042.xml en Cats_Test2042.txt dans train
Convert

In [47]:
# Objectif : S'assurer que les dossiers images/ et annotations/ existent
dataset_path = '/content/dataset'
images_path = os.path.join(dataset_path, 'images')
labels_path = os.path.join(dataset_path, 'labels')  # Corrected to 'labels'
classes_path = os.path.join(dataset_path, 'classes.txt')

# Vérifier l'existence des dossiers
if not os.path.exists(images_path) or not os.path.exists(labels_path):
    raise FileNotFoundError("Dossier images/ ou labels/ manquant. Vérifiez le dataset.")

# Lister les fichiers pour inspection
print("Contenu du dossier dataset :")
for root, dirs, files in os.walk(dataset_path):
    print(f"Dossier : {root}")
    print(f"Fichiers : {files[:5]}")
    print(f"Sous-dossiers : {dirs}")
    print("-" * 50)

# Créer classes.txt si absent (2 classes : cat, dog)
if not os.path.exists(classes_path):
    classes = ['cat', 'dog']
    with open(classes_path, 'w') as f:
        for cls in classes:
            f.write(f"{cls}\n")
    print(f"Fichier classes.txt créé avec les classes : {classes}")
else:
    with open(classes_path, 'r') as f:
        classes = [line.strip() for line in f.readlines()]
    print(f"Classes trouvées : {classes}")

# Renommer le dossier annotations/ en labels/ pour correspondre aux attentes de YOLOv9
new_labels_path = os.path.join(dataset_path, 'labels')
if not os.path.exists(new_labels_path):
    os.rename(labels_path, new_labels_path)
    labels_path = new_labels_path
    print(f"Dossier renommé : {labels_path}")
else:
    print(f"Dossier {new_labels_path} existe déjà.")

Contenu du dossier dataset :
Dossier : /content/dataset
Fichiers : ['classes.txt']
Sous-dossiers : ['labels', 'images', 'annotations']
--------------------------------------------------
Dossier : /content/dataset/labels
Fichiers : ['Cats_Test87.xml', 'Cats_Test3411.xml', 'Cats_Test105.xml', 'Cats_Test1768.xml', 'Cats_Test2076.xml']
Sous-dossiers : ['train', 'val', 'test']
--------------------------------------------------
Dossier : /content/dataset/labels/train
Fichiers : ['Cats_Test3244.txt', 'Cats_Test2396.txt', 'Cats_Test2570.txt', 'Cats_Test3620.txt', 'Cats_Test589.txt']
Sous-dossiers : []
--------------------------------------------------
Dossier : /content/dataset/labels/val
Fichiers : ['Cats_Test3045.txt', 'Cats_Test720.txt', 'Cats_Test1412.txt', 'Cats_Test1723.txt', 'Cats_Test1792.txt']
Sous-dossiers : []
--------------------------------------------------
Dossier : /content/dataset/labels/test
Fichiers : ['Cats_Test1941.txt', 'Cats_Test445.txt', 'Cats_Test520.txt', 'Cats_Test25

In [48]:
# Objectif : Vérifier les dossiers train/val/test après conversion des annotations
# Vérifier le contenu des dossiers images et labels
labels_path = os.path.join(dataset_path, 'labels')  # Corrected to 'labels'
for split in ['train', 'val', 'test']:
    img_dir = os.path.join(images_path, split)
    lbl_dir = os.path.join(labels_path, split)

    img_count = len([f for f in os.listdir(img_dir) if f.endswith('.png')])
    lbl_count = len([f for f in os.listdir(lbl_dir) if f.endswith('.txt')])

    print(f"Dossier {split} : {img_count} images, {lbl_count} annotations")

Dossier train : 2580 images, 2580 annotations
Dossier val : 553 images, 553 annotations
Dossier test : 553 images, 553 annotations


In [49]:
# Objectif : Télécharger le code officiel de YOLOv9
!git clone https://github.com/WongKinYiu/yolov9.git
%cd /content/yolov9

# Installer les dépendances de YOLOv9
!pip install -r requirements.txt -q

fatal: destination path 'yolov9' already exists and is not an empty directory.
/content/yolov9


In [50]:
# Objectif : Créer data.yaml pour spécifier les chemins et classes
data_yaml = f"""
train: {dataset_path}/images/train
val: {dataset_path}/images/val
test: {dataset_path}/images/test
train_labels: {dataset_path}/labels/train  # Corrected path
val_labels: {dataset_path}/labels/val     # Corrected path
test_labels: {dataset_path}/labels/test   # Corrected path
nc: {len(classes)}  # Nombre de classes
names: {classes}  # Liste des classes
"""

with open('data.yaml', 'w') as f:
    f.write(data_yaml)

# Vérifier le contenu de data.yaml
!cat data.yaml


train: /content/dataset/images/train
val: /content/dataset/images/val
test: /content/dataset/images/test
train_labels: /content/dataset/labels/train  # Corrected path
val_labels: /content/dataset/labels/val     # Corrected path
test_labels: /content/dataset/labels/test   # Corrected path
nc: 2  # Nombre de classes
names: ['cat', 'dog']  # Liste des classes


In [51]:
# Objectif: vérifier si le bon fichier d'hyperparamètres est présent
hyp_dir = 'data/hyps'
if not os.path.exists(hyp_dir):
    os.makedirs(hyp_dir)

print("Fichiers dans data/hyps :")
!ls {hyp_dir}

hyp_file = os.path.join(hyp_dir, 'hyp.scratch-low.yaml')
if os.path.exists(hyp_file):
    os.remove(hyp_file)

hyp_content = """
lr0: 0.01
lrf: 0.01
momentum: 0.937
weight_decay: 0.0005
warmup_epochs: 3.0
warmup_momentum: 0.8
warmup_bias_lr: 0.1
box: 0.05
cls: 0.5
cls_pw: 1.0
obj: 1.0
obj_pw: 1.0
iou_t: 0.20
anchor_t: 4.0
fl_gamma: 0.0
hsv_h: 0.015
hsv_s: 0.7
hsv_v: 0.4
degrees: 0.0
translate: 0.1
scale: 0.5
shear: 0.0
perspective: 0.0
flipud: 0.0
fliplr: 0.5
mosaic: 1.0
mixup: 0.0
copy_paste: 0.0
albumentations:
  size: [416, 416]  # Changé de [640, 640] à [416, 416]
  scale: [0.5, 1.0]
  hsv_h: 0.015
  hsv_s: 0.7
  hsv_v: 0.4
  degrees: 0.0
  translate: 0.1
  shear: 0.0
  perspective: 0.0
  flipud: 0.0
  fliplr: 0.5
  mosaic: 1.0
  mixup: 0.0
  copy_paste: 0.0
"""
with open(hyp_file, 'w') as f:
    f.write(hyp_content.strip())
print(f"Fichier {hyp_file} créé avec des hyperparamètres par défaut.")

Fichiers dans data/hyps :
hyp.scratch-high.yaml  hyp.scratch-low.yaml
Fichier data/hyps/hyp.scratch-low.yaml créé avec des hyperparamètres par défaut.


In [52]:
# Vérifier si le fichier yolov9-c.pt existe, sinon le télécharger
weights_file = 'yolov9-c.pt'
if not os.path.exists(weights_file):
    print(f"Fichier {weights_file} non trouvé. Téléchargement en cours...")
    # Lien officiel pour yolov9-c.pt (à partir du dépôt ou d'un lien fourni par les auteurs)
    !wget https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-c.pt -O {weights_file}
    if os.path.exists(weights_file):
        print(f"Fichier {weights_file} téléchargé avec succès.")
    else:
        print(f"Échec du téléchargement de {weights_file}. Entraînement sans poids pré-entraînés.")
        weights_file = ''  # Entraîner à partir de zéro si le téléchargement échoue
else:
    print(f"Fichier {weights_file} déjà présent.")

Fichier yolov9-c.pt déjà présent.


In [53]:
print(os.listdir())
for i in os.walk('.'):
  print(i)

['detect.py', 'hubconf.py', 'LICENSE.md', 'data.yaml', 'classify', 'runs', 'val.py', 'segment', 'dog-and-cat-detection.zip', 'val_triple.py', 'models', 'tools', 'train_dual.py', 'README.md', 'benchmarks.py', 'train.py', 'panoptic', '.git', '__pycache__', 'train_triple.py', 'figure', 'yolov9-c.pt', 'kaggle.json', 'kaggle (2).json', 'kaggle (1).json', 'yolov9', 'val_dual.py', 'data', 'export.py', 'utils', 'detect_dual.py', 'scripts', 'requirements.txt']
('.', ['classify', 'runs', 'segment', 'models', 'tools', 'panoptic', '.git', '__pycache__', 'figure', 'yolov9', 'data', 'utils', 'scripts'], ['detect.py', 'hubconf.py', 'LICENSE.md', 'data.yaml', 'val.py', 'dog-and-cat-detection.zip', 'val_triple.py', 'train_dual.py', 'README.md', 'benchmarks.py', 'train.py', 'train_triple.py', 'yolov9-c.pt', 'kaggle.json', 'kaggle (2).json', 'kaggle (1).json', 'val_dual.py', 'export.py', 'detect_dual.py', 'requirements.txt'])
('./classify', [], ['predict.py', 'val.py', 'train.py'])
('./runs', ['train', '

In [54]:
# Objectif : Lancer l'entraînement avec des hyperparamètres adaptés au T4
start_time = time.time()
# !python train.py --noval --weights {weights_file} --cfg models/detect/yolov9-c.yaml --data data.yaml --hyp data/hyps/hyp.scratch-low.yaml --epochs 15 --batch-size 8 --img 416 --device 0 --name exp --exist-ok

!python train.py \
  --data data.yaml \
  --cfg models/detect/yolov9-c.yaml \
  --weights yolov9-c.pt \
  --imgsz 416 \
  --batch-size 8 \
  --epochs 1 \
  --device 0 \
  --optimizer SGD \
  --hyp data/hyps/hyp.scratch-low.yaml \
  --evolve \
  --noplots \
  --project runs/train \
  --name yolov9_exp_evolve \
  --exist-ok


training_time = (time.time() - start_time) / 60
print(f"Temps d'entraînement : {training_time:.2f} minutes")

# Copier les résultats de l'entraînement sur Google Drive
# Vérifier si le dossier runs/train/exp existe, contient des fichiers, et si l'entraînement a réussi
train_exp_dir = 'runs/train/exp'
weights_dir = os.path.join(train_exp_dir, 'weights')
if os.path.exists(train_exp_dir) and os.path.exists(weights_dir) and os.listdir(weights_dir):
    !cp -r {train_exp_dir} {output_dir}/train_exp_{datetime.now().strftime('%Y%m%d_%H%M%S')}
    print(f"Résultats de l'entraînement copiés dans {output_dir}")
else:
    print(f"Dossier {train_exp_dir} ou sous-dossier weights/ non trouvé ou vide. L'entraînement a probablement échoué.")

2025-04-26 14:07:44.342299: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745676464.418783   34131 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745676464.435560   34131 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-26 14:07:44.486178: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb:

In [55]:
# Objectif : Calculer les métriques (mAP) sur le dataset de validation
!python val.py --weights runs/train/exp/weights/best.pt --data data.yaml --img 640 --device 0 --task val

val: data=data.yaml, weights=['runs/train/exp/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.7, max_det=300, task=val, device=0, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False, min_items=0
YOLO 🚀 v0.1-104-g5b1ea9a Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)

/content/yolov9/models/experimental.py:243: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitr

In [56]:
# Objectif : Tester sur des images de test et visualiser les résultats
test_dir = os.path.join(dataset_path, 'images/test')
if not os.path.exists(test_dir):
    print(f"Dossier de test {test_dir} non trouvé. Utilisation de quelques images de validation.")
    test_dir = os.path.join(dataset_path, 'images/val')

# Lancer la détection
!python detect.py --weights runs/train/exp/weights/best.pt --source {test_dir} --img 640 --conf 0.25 --device 0 --save-txt --save-conf --name test_exp

detect: weights=['runs/train/exp/weights/best.pt'], source=/content/dataset/images/test, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=0, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=test_exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLO 🚀 v0.1-104-g5b1ea9a Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)

/content/yolov9/models/experimental.py:243: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default va

In [57]:
# Visualiser quelques résultats
output_test_dir = 'runs/detect/test_exp'
if os.path.exists(output_test_dir):
    test_images = [os.path.join(output_test_dir, f) for f in os.listdir(output_test_dir) if f.endswith(('.jpg', '.png'))][:3]
    plt.figure(figsize=(15, 5))
    for i, img_path in enumerate(test_images):
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.subplot(1, 3, i+1)
        plt.imshow(img)
        plt.axis('off')
        plt.title(f"Résultat {i+1}")
    plt.show()
else:
    print("Aucun résultat de détection trouvé.")


<Figure size 1500x500 with 0 Axes>

In [58]:
# Objectif : Exporter le modèle au format ONNX pour une utilisation future
!python export.py --weights runs/train/exp/weights/best.pt --img 640 --include onnx --device 0

# Copier le modèle exporté sur Google Drive
onnx_path = 'runs/train/exp/weights/best.onnx'
if os.path.exists(onnx_path):
    !cp {onnx_path} {output_dir}/yolov9_best_{datetime.now().strftime('%Y%m%d_%H%M%S')}.onnx
    print(f"Modèle ONNX exporté et sauvegardé dans {output_dir}")
else:
    print("Erreur lors de l'exportation ONNX.")


IndentationError: unexpected indent (<ipython-input-58-3c20daa33e39>, line 3)

In [ ]:
# Objectif : Évaluer les performances (mAP, FPS, temps d'inférence)
# 1. Récupérer mAP depuis l'évaluation
map_file = 'runs/val/exp/val_metrics.txt'  # Chemin fictif, ajustez selon YOLOv9
if os.path.exists(map_file):
    with open(map_file, 'r') as f:
        map_metrics = f.read()
    print(f"Métriques mAP : {map_metrics}")
else:
    print("Fichier de métriques mAP non trouvé. Vérifiez runs/val/.")

# 2. Mesurer le FPS (images par seconde)
start_time = time.time()
!python detect.py \
    --weights runs/train/exp/weights/best.pt \
    --source {test_dir} \
    --img 640 \
    --conf 0.25 \
    --device 0 \
    --name bench_exp
inference_time = time.time() - start_time
num_images = len([f for f in os.listdir(test_dir) if f.endswith(('.jpg', '.png'))])
fps = num_images / inference_time if inference_time > 0 else 0
print(f"Temps d'inférence total : {inference_time:.2f} secondes")
print(f"Nombre d'images testées : {num_images}")
print(f"FPS (images par seconde) : {fps:.2f}")

# 3. Comparaison avec une baseline (YOLOv5 pour référence)
baseline_map = 0.50  # mAP typique pour YOLOv5 sur un dataset similaire
baseline_fps = 45.0  # FPS typique sur T4
print(f"Comparaison avec YOLOv5 (baseline) :")
print(f"- mAP YOLOv9 : À déterminer après entraînement (vs {baseline_map} pour YOLOv5)")
print(f"- FPS YOLOv9 : {fps:.2f} (vs {baseline_fps} pour YOLOv5)")

# 4. Visualisation des performances
metrics_df = pd.DataFrame({
    'Modèle': ['YOLOv9', 'YOLOv5 (baseline)'],
    'FPS': [fps, baseline_fps],
    'mAP': [0, baseline_map]  # Remplacer 0 par la vraie valeur mAP après entraînement
})
plt.figure(figsize=(8, 5))
sns.barplot(x='Modèle', y='FPS', data=metrics_df)
plt.title('Comparaison des FPS')
plt.show()

In [ ]:
# Objectif : S'assurer que tous les résultats sont sur Google Drive
!cp -r runs/detect/test_exp {output_dir}/test_exp_{datetime.now().strftime('%Y%m%d_%H%M%S')}
print(f"Tous les résultats ont été sauvegardés dans {output_dir}")